<a href="https://colab.research.google.com/github/vicente-gonzalez-ruiz/PRMC-experimentos/blob/master/src/shift_invariance_in_OLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shift invariance in LP

Is Laplacian Pyramid shift invariant as it is claimed in [Shift-invariance in the Discrete Wavelet
Transform](https://eprints.qut.edu.au/114753/1/Bradley_DICTA03.pdf)?

In [ ]:
%matplotlib inline
from load_and_display import *
#import cv2
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

In [ ]:
%%bash

# Create the two images ("/tmp/moving_circle_000.png" and "/tmp/moving_circle_001.png") with a circle
# of diameter 10 that moves from coordinate (20,30) to (21,30).
../tools/moving_circle.sh -w 128 -h 128 -x 20 -y 30 -d 10 -f 2

In [ ]:
moving_circle_000 = cv2.imread("/tmp/moving_circle_000.png")

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/moving_circle_000.png", fontsize=20)
plt.imshow(moving_circle_000)

In [ ]:
moving_circle_000 = cv2.imread("/tmp/moving_circle_001.png")

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/moving_circle_001.png", fontsize=20)
plt.imshow(moving_circle_000)

In [ ]:
%%bash

# The images are in the RGB domain and we work in the YUV domain, so we convert them.
../tools/RGB2YUV.py -i /tmp/moving_circle_000.png -o /tmp/YUV_moving_circle_000.png
../tools/RGB2YUV.py -i /tmp/moving_circle_001.png -o /tmp/YUV_moving_circle_001.png

In [ ]:
# Plot the first part (from pixel 0 to 39) of the horizontal line (row 35) of the luminance.
YUV_moving_circle_000 = cv2.imread("/tmp/YUV_moving_circle_000.png")
YUV_moving_circle_001 = cv2.imread("/tmp/YUV_moving_circle_001.png")
plt.plot(YUV_moving_circle_000[35, 0:40, 0])
plt.show()
plt.plot(YUV_moving_circle_001[35, 0:40, 0])
plt.show()

In [ ]:
# Generate Gaussian pyramid (of 2 levels) for both images
# (https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_pyramids/py_pyramids.html)

levels = 2

G_000 = YUV_moving_circle_000[:,:,0].copy()
G_001 = YUV_moving_circle_001[:,:,0].copy()
gp_000 = [G_000]
gp_001 = [G_001]
for i in range(levels):
    G_000 = cv2.pyrDown(G_000)
    G_001 = cv2.pyrDown(G_001)
    gp_000.append(G_000)
    gp_001.append(G_001)

In [ ]:
# Generate Laplacian Pyramid for both images.
lp_000 = [gp_000[levels-1]]
lp_001 = [gp_001[levels-1]]
for i in range(levels-1, 0, -1):
    GE_000 = cv2.pyrUp(gp_000[i])
    GE_001 = cv2.pyrUp(gp_001[i])
    L_000 = cv2.subtract(gp_000[i-1], GE_000)
    L_001 = cv2.subtract(gp_001[i-1], GE_001)
    lp_000.append(L_000)
    lp_001.append(L_001)

## Is $[L]$ subband shift invariant?
A priori, aliasing makes impossible that $[L]$ is shift-invariant.

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/level 1 (image 000)", fontsize=20)
plt.imshow(cv2.pyrUp(lp_000[0]), cmap="gray")

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/level 1 (image 001)", fontsize=20)
plt.imshow(cv2.pyrUp(lp_001[0]), cmap="gray")

In [ ]:
plt.plot(cv2.pyrUp(lp_000[0])[35, 0:40])
plt.show()
plt.plot(cv2.pyrUp(lp_001[0])[35, 0:40])
plt.show()

But it is quite close to being.

## Is $[H]$ subband shift invariant?
If $[L]$ is almost shift invariant,
$$
[H] = I - [L]
$$
where $I$ is the original image, should also be near shift invariant.

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/000_H_0", fontsize=20)
plt.imshow(lp_000[1], cmap="gray")

In [ ]:
plt.figure(figsize=(10,10))
plt.title("/tmp/000_H_1", fontsize=20)
plt.imshow(lp_001[1], cmap="gray")

In [ ]:
plt.figure(figsize=(10,10))
plt.title("diff", fontsize=20)
plt.imshow(lp_001[1]-lp_000[1], cmap="gray")

In [ ]:
plt.plot(lp_000[1][35, 0:40])
plt.show()
plt.plot(lp_001[1][35, 0:40])
plt.show()

In [ ]:
plt.plot(lp_001[1][35, 0:40]-np.roll(lp_000[1][35, 0:40],1)+128)
plt.show()

The $[H]$ subband is also quite close to being shift invariant.